In [1]:
using Pkg

Pkg.activate("../Project.toml")

using Random
import .Iterators: cycle, Cycle, take

using Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!
using JLD2
using CUDA

 Activating environment at `~/Desktop/MSc/repos/yolo-julia/Project.toml`


In [3]:
function Knet.KnetArray(x::CuArray{T,N}) where {T,N}
    p = Base.bitcast(Knet.Cptr, pointer(x))
    k = Knet.KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
    KnetArray{T,N}(k, size(x))
end

In [2]:
include("../coco2014.jl")
x, y = load_data(
    "../dataset/images/overfit",
    "../val_labels.json",
    class_file="../class_mappings.json"
);

dtrn = minibatch(x[:, :, :, 1:6], y[1:6], 2; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(x[:, :, :, 10:11], y[10:11], 2; xsize = (416,416,3,:), xtype=Knet.atype());

# xx, yy = first(dtrn);

In [26]:
include("../models.jl")

Random.seed!(1)
darknet = Darknet("../yolov3.cfg"; verbose=false);


@diff a = darknet(xx, yy)

# a = darknet(xx, yy)

T(572.3726812532891)

In [ ]:
bce(scores, labels; average=true)

In [21]:
bce(out[5, :, :, :, :][:], convert(Array{Integer}, tobj)[:])

0.77960855f0

In [ ]:
tobj

In [56]:
min.(max.(exp.(ps[3:4, :]), 0), 1000)  # custom clamp

2×11 KnetArray{Float32,2}:
 0.894371  0.894262  0.628493  0.658711  …  1.00179   0.760402  0.781808
 1.01587   1.02907   1.0577    1.04478      0.919599  0.973357  1.02486

3×6 Array{Integer,2}:
 0  0  1  1  1  1
 1  0  1  1  1  1
 1  0  0  0  1  0

In [39]:
an = convert(Array{Bool}, a)

3×6 Array{Bool,2}:
 0  0  1  1  1  1
 1  0  1  1  1  1
 1  0  0  0  1  0

In [19]:
vcat(an'...)

18-element Array{Int64,1}:
 0
 0
 1
 1
 1
 1
 1
 0
 1
 1
 1
 1
 1
 0
 0
 0
 1
 0

In [3]:
function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss[end])
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end



train! (generic function with 2 methods)

In [4]:

dtrn = minibatch(x[:, :, :, 1:6], y[1:6], 1; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(x[:, :, :, 10:11], y[10:11], 1; xsize = (416,416,3,:), xtype=Knet.atype());
include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

model = darknet

iters, trnloss, trnacc, tstloss, tstacc = train!(
    model, dtrn, dtst; 
    period=1, iters=520, lr=1e-2, optimizer=adam!);

Iter: 0 573.352769022327
Iter: 1 3.475064706126181e13
Iter: 2 8.643858860833297e19
Iter: 3 2.130118982840143e21
Iter: 4 4.51618392965065e21
Iter: 5 2.46488088175084e21
Iter: 6 3.048158987135386e20

Stacktrace:
 [1] Array at ./boot.jl:418 [inlined]
 [2] Array at ./boot.jl:423 [inlined]
 [3] similar at ./abstractarray.jl:675 [inlined]
 [4] similar at ./abstractarray.jl:674 [inlined]
 [5] similar at ./broadcast.jl:197 [inlined]
 [6] similar at ./broadcast.jl:196 [inlined]
 [7] copy(::Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{4},Nothing,typeof(*),Tuple{Array{Float32,1},Array{Float32,4}}}) at ./broadcast.jl:862
 [8] forw(::Function, ::Function, ::Vararg{Any,N} where N; kwargs::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /home/bcs/.julia/packages/AutoGrad/VFrAv/src/core.jl:69
 [9] forw at /home/bcs/.julia/packages/AutoGrad/VFrAv/src/core.jl:65 [inlined]
 [10] broadcasted(::typeof(*), ::Array{Float32,1}, ::AutoGrad.Result{Array{Float32,4}}) at ./

LoadError: InterruptException:

In [22]:
@load "first.jld2" darknet

1-element Array{Symbol,1}:
 :darknet

In [23]:
@diff a = darknet(x[:, :, :, 1:2], y[1:2])

T(5.974961309897038)

In [ ]:
plot(iters, trnloss, label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, tstloss, label="test"))

plot(iters, round.(1 .- trnacc, digits=2), label="train", xlabel="Iterations", ylabel="Misclassification error")
display(plot!(iters, round.(1 .- tstacc, digits=2), label="test"))

In [ ]:

include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

l = darknet(xx, yy)


In [ ]:
sum(1.0 .- giou)

In [ ]:
inter = clamp.(min.(b1_x2, b2_x2) - max.(b1_x1, b2_x1), 0, 1e10) .* 
clamp.(min.(b1_y2, b2_y2) - max.(b1_y1, b2_y1), 0, 1e10);


In [ ]:
pbox[1, :] .- pbox[3, :] ./ 2

In [ ]:
clamp.(exp.(scores), 0, 1e3)

In [ ]:

b = Integer.(t[:, 1])
c = Integer.(t[:, 2])
gxy = t[:, 3:4]
gwh = t[:, 5:6]

In [ ]:
size(yolo_out[1])

In [ ]:
j

In [ ]:
j'

In [ ]:
tt[jj, :]

In [ ]:
tt = repeat(t, 3)

In [ ]:
tt[j]

In [ ]:
at'[jj]

In [ ]:
targets_reshaped .* gain

In [ ]:
wh1r = reshape(wh1, (:, 1, 2))

In [ ]:
wh12 = reshape(prod(wh1r, dims=3), (:, 1))

In [ ]:
wh2

In [ ]:
wh2r = reshape(wh2, (1, :, 2))

In [ ]:
wh22 = reshape(prod(wh2r, dims=3), (1, :))

In [ ]:
wh12 .+ wh22

In [ ]:
reshape(prod(a, dims=3), (3, 13))

In [ ]:
inter = build_targets(yolo_out, yy, darknet)


In [ ]:
targets = nothing 

for (i, t) in enumerate(yy[2:end])
    println(i)

end
gg = hcat(yy[2]...)
# gg = vcat(ones(1, 2), gg)

In [ ]:
length(yy[2])


In [ ]:
include("utils/utils.jl")

